<a href="https://colab.research.google.com/github/JashodaKumawat123/Mlops/blob/main/ML_pipelines_with_ZenMl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from IPython import get_ipython
from IPython.display import display
# %%
%pip install "zenml[server]"
!zenml integration install sklearn -y
# %pip install pyparsing==2.4.2 # Removed to avoid potential conflicts

import IPython

#automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93

{'status': 'ok', 'restart': True}

In [ ]:
%pip install pyparsing

In [ ]:
NGROK_TOKEN ='2u24HGvTxaSlN7OGDQuM6NMTUvC_2pST1DMKHZtC5vphuTifh' #TODO: set your ngrok token if you are working on Colab

In [ ]:
from zenml.environment import Environment
#install and authenticate ngrok

if Environment.in_google_colab():
  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!rm -rf .zenml
!zenml init

Initializing the ZenML global configuration version to 0.75.0
Creating database tables
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializin

In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test()-> None:
  digits= load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train,y_test=train_test_split(data, digits.target,test_size=0.2,shuffle=False)
  model =SVC(gamma=0.001)
  model.fit(X_train,y_train)
  test_acc=model.score(X_test,y_test)
  print(f"Accuracy: {test_acc}")
train_test()



INFO:numexpr.utils:NumExpr defaulting to 2 threads.


NumExpr defaulting to 2 threads.
Accuracy: 0.9583333333333334


In [ ]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer()->Tuple[
    Annotated[np.ndarray,"X_train"],
    Annotated[np.ndarray,"X_test"],
    Annotated[np.ndarray,"y_train"],
    Annotated[np.ndarray,"y_test"],]:
    digits= load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train,y_test=train_test_split(data, digits.target,test_size=0.2,shuffle=False)
    return X_train, X_test, y_train,y_test
@step
def svc_trainer(
    X_train: np.ndarray,
    # X_test: np.ndarray,
    y_train: np.ndarray,
    # y_test: np.ndarray,
)->ClassifierMixin:
    model =SVC(gamma=0.001)
    model.fit(X_train,y_train)
    return model

@step
def evaluator(
    X_test :np.ndarray,
    y_test:np.ndarray,
    model:ClassifierMixin
)->float:
    test_acc=model.score(X_test,y_test)
    print(f"Accuracy: {test_acc}")
    return test_acc



In [ ]:
from zenml import pipeline

@pipeline
def digits_pipeline():
    X_train, X_test, y_train,y_test=importer()
    model=svc_trainer(X_train=X_train,y_train=y_train)
    evaluator(X_test=X_test,y_test=y_test,model=model)

In [ ]:
digits_svc_pipeline =digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Registered new pipeline: digits_pipeline.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step importer has started.
Step importer has finished in 6.798s.
Step svc_trainer has started.
Step svc_trainer has finished in 1.369s.
Step evaluator has started.
Accuracy: 0.9583333333333334
Step evaluator has finished in 0.342s.
Pipeline run has finished in 8.808s.


In [ ]:
from pyngrok import ngrok

In [ ]:
pip install pyngrok

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `ngrok authtoken <your_authtoken>'


In [ ]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
    if Environment.in_google_colab():
        !pip install pyngrok  # Install pyngrok if in Colab
        from pyngrok import ngrok

        # Set the authtoken (replace with your actual token)
        from pyngrok import ngrok

        public_url = ngrok.connect(port)
        print(f"\x1b[31mIn Colab, use this url instead: {public_url}!\x1b[0m")
        !zenml up --blocking --port {port}
    else:
        !zenml up --port {port}

start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-5d2fe8cb-0d79-49d4-bd1c-5f9392b43e84


Opening tunnel named: http-8237-5d2fe8cb-0d79-49d4-bd1c-5f9392b43e84


INFO:pyngrok.process.ngrok:t=2025-03-09T05:27:40+0000 lvl=info msg="no configuration paths supplied"


t=2025-03-09T05:27:40+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2025-03-09T05:27:40+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

In Colab, use this url instead: NgrokTunnel: "https://f71c-34-148-30-70.ngrok-free.app" -> "http://localhost:8237"!
t=2025-03-09T05:27:40+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-5d2fe8cb-0d79-49d4-bd1c-5f9392b43e84 addr=http://localhost:8237 url=https://f71c-34-148-30-70.ngrok-free.app


INFO:pyngrok.process.ngrok:t=2025-03-09T05:27:40+0000 lvl=info msg=end pg=/api/tunnels id=b91c2c93138de0c1 status=201 dur=45.776665ms


t=2025-03-09T05:27:40+0000 lvl=info msg=end pg=/api/tunnels id=b91c2c93138de0c1 status=201 dur=45.776665ms
The `zenml up` command is deprecated and will be removed in a future release. Please use the `zenml 
login --local` command instead.
Calling `zenml login --local`...
The local ZenML dashboard is about to deploy in a blocking process.
Deploying a local daemon ZenML server.
Not writing the global configuration to disk in a ZenML server environment.
Initializing the ZenML global configuration version to 0.75.0
Not writing the global configuration to disk in a ZenML server environment.
Starting ZenML Server as blocking process... press CTRL+C once to stop it.
INFO:     Started server process [1156]
INFO:     Waiting for application startup.
Not writing the global configuration to disk in a ZenML server environment.
Not writing the global configuration to disk in a ZenML server environment.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8237 (Pres

INFO:pyngrok.process.ngrok:t=2025-03-09T05:46:54+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


t=2025-03-09T05:46:54+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
INFO:     Shutting down
INFO:     Finished server process [1156]
ERROR:    Traceback (most recent call last):
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "uvloop/loop.pyx", line 1512, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1505, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1379, in uvloop.loop.Loop.run_forever
  File "uvloop/loop.pyx", line 557, in uvloop.loop.Loop._run
  File "uvloop/loop.pyx", line 476, in uvloop.loop.Loop._on_idle
  File "uvloop/cbhandles.pyx", line 83, in uvloop.loop.Handle._run
  File "uvloop/cbhandles.pyx", line 63, in uvloop.loop.Handle._run
  File "/usr/local/

In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
get_ipython().run_line_magic('pip', 'install "zenml[server]"')
get_ipython().system('zenml integration install sklearn -y')
# %pip install pyparsing==2.4.2 # Removed to avoid potential conflicts

import IPython

#automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)
# %%
get_ipython().run_line_magic('pip', 'install pyparsing')
# %%
NGROK_TOKEN ='2u24HGvTxaSlN7OGDQuM6NMTUvC_2pST1DMKHZtC5vphuTifh'  # Replace with your actual Ngrok token
# %%
from zenml.environment import Environment
#install and authenticate ngrok

if Environment.in_google_colab():
  get_ipython().system('pip install pyngrok')
  get_ipython().system('ngrok authtoken {NGROK_TOKEN}')
# %%
get_ipython().system('rm -rf .zenml')
get_ipython().system('zenml init')
# %%
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import joblib  # Import joblib for caching

def train_test()-> None:
  digits= load_digits()
  data = digits.images.reshape((len(digits.images), -1))
  X_train, X_test, y_train,y_test=train_test_split(data, digits.target,test_size=0.2,shuffle=False)
  model =SVC(gamma=0.001)
  model.fit(X_train,y_train)
  test_acc=model.score(X_test,y_test)
  print(f"Accuracy: {test_acc}")
train_test()


# %%
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer()->Tuple[
    Annotated[np.ndarray,"X_train"],
    Annotated[np.ndarray,"X_test"],
    Annotated[np.ndarray,"y_train"],
    Annotated[np.ndarray,"y_test"],]:

    try:
        X_train, X_test, y_train, y_test = joblib.load('cached_data.pkl')
    except FileNotFoundError:
        digits = load_digits()
        data = digits.images.reshape((len(digits.images), -1))
        X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.2, shuffle=False)
        joblib.dump((X_train, X_test, y_train, y_test), 'cached_data.pkl')

    return X_train, X_test, y_train, y_test

@step
def svc_trainer(
    X_train: np.ndarray,
    # X_test: np.ndarray,
    y_train: np.ndarray,
    # y_test: np.ndarray,
)->ClassifierMixin:
    model =SVC(gamma=0.001)
    model.fit(X_train,y_train)
    return model

@step
def evaluator(
    X_test :np.ndarray,
    y_test:np.ndarray,
    model:ClassifierMixin
)->float:
    test_acc=model.score(X_test,y_test)
    print(f"Accuracy: {test_acc}")
    return test_acc


# %%
from zenml import pipeline

@pipeline
def digits_pipeline():
    X_train, X_test, y_train,y_test=importer()
    model=svc_trainer(X_train=X_train,y_train=y_train)
    evaluator(X_test=X_test,y_test=y_test,model=model)
# %%
digits_svc_pipeline =digits_pipeline()
# %%

# %%
from pyngrok import ngrok
# %%
get_ipython().system('pip install pyngrok')
# %%

# %%
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
    if Environment.in_google_colab():
        get_ipython().system('pip install pyngrok  # Install pyngrok if in Colab')
        from pyngrok import ngrok

        # Set the authtoken (replace with your actual token)
        from pyngrok import ngrok

        public_url = ngrok.connect(port)
        print(f"\x1b[31mIn Colab, use this url instead: {public_url}!\x1b[0m")
        get_ipython().system('zenml up --blocking --port {port}')
    else:
        get_ipython().system('zenml up --port {port}')

start_zenml_dashboard()
# %%

⠸ Installing integrations...
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Found existing ZenML repository at path '/content'.
⠋ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.

Accuracy: 0.9583333333333334
Initiating a new run for the pipeline: digits_pipeline.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step importer has started.
Step importer has finished in 4.942s.
Step svc_trainer has started.
Step svc_trainer has finished in 1.767s.
Step evaluator has started.
Accuracy: 0.9583333333333334
Step evaluator has finished in 0.387s.
Pipeline run has finished in 7.365s.


INFO:pyngrok.ngrok:Opening tunnel named: http-8237-915f4e54-4b3b-4a9a-b2c4-0806b0af4971


Opening tunnel named: http-8237-915f4e54-4b3b-4a9a-b2c4-0806b0af4971


INFO:pyngrok.process.ngrok:t=2025-03-09T05:48:50+0000 lvl=info msg="no configuration paths supplied"


t=2025-03-09T05:48:50+0000 lvl=info msg="no configuration paths supplied"


...

t=2025-03-09T05:54:04+0000 lvl=info msg="join connections" obj=join id=7c2c20dfcaa5 l=127.0.0.1:8237 r=[2401:4900:5f2d:237a:dcbe:c284:2908:aad5]:57430


INFO:pyngrok.process.ngrok:t=2025-03-09T05:54:04+0000 lvl=info msg="join connections" obj=join id=26945d437214 l=127.0.0.1:8237 r=[2401:4900:5f2d:237a:dcbe:c284:2908:aad5]:57430


t=2025-03-09T05:54:04+0000 lvl=info msg="join connections" obj=join id=26945d437214 l=127.0.0.1:8237 r=[2401:4900:5f2d:237a:dcbe:c284:2908:aad5]:57430


INFO:pyngrok.process.ngrok:t=2025-03-09T05:54:04+0000 lvl=info msg="join connections" obj=join id=2acf81c5c943 l=127.0.0.1:8237 r=[2401:4900:5f2d:237a:dcbe:c284:2908:aad5]:57430


t=2025-03-09T05:54:04+0000 lvl=info msg="join connections" obj=join id=2acf81c5c943 l=127.0.0.1:8237 r=[2401:4900:5f2d:237a:dcbe:c284:2908:aad5]:57430
INFO:     2401:4900:5f2d:237a:dcbe:c284:2908:aad5:0 - "GET /api/v1/current-user HTTP/1.1" 200 OK
INFO:     2401:4900:5f2d:237a:dcbe:c284:2908:aad5:0 - "GET /api/v1/onboarding_state HTTP/1.1" 200 OK
INFO:     2401:4900:5f2d:237a:dcbe:c284:2908:aad5:0 - "GET /api/v1/info HTTP/1.1" 200 OK
INFO:     2401:4900:5f2d:237a:dcbe:c284:2908:aad5:0 - "GET /api/v1/pipeline_deployments/6f01777f-37dc-4a41-b63d-2492c5661b62 HTTP/1.1" 200 OK
INFO:     2401:4900:5f2d:237a:dcbe:c284:2908:aad5:0 - "GET /api/v1/settings HTTP/1.1" 200 OK
INFO:     2401:4900:5f2d:237a:dcbe:c284:2908:aad5:0 - "GET /api/v1/runs/4740bcd5-88bd-48b1-96be-4f9104951983 HTTP/1.1" 200 OK
